In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib as plt
import pprint
import json

In [2]:
class Main():
    def __init__(self):
        self.subject_list = ["takeda", "shintani", 
                             "okamoto", "kyomine", 
                             "horie", "hasegawa", 
                             "kawakata", "tanaka",
                             "yoshida", "ikari",
                             "huruki", "horii"]
        self.parameter = ["radius", "theta",
                          "valence", "arousal",
                          "sliderThetaSub", "sliderVertexNum", 
                          "sliderLengthCoefficient", "sliderAmplitude",
                          "sliderThetaBeat", "sliderHue"]
        self.order_num = 17
        self.map_hue = {}
        self.df_overall_hue = None
        self.df_overall_nohue = None
        self.df_each_order_hue = []
        self.df_each_order_average_hue = None
        self.df_each_order_nohue = []
        self.df_each_order_average_nohue = None
        self.df_each_parameter_hue = None
        self.df_each_parameter_nohue = None
        self.df_list_hue = []
        self.df_list_nohue = []
        self.minBrightness = -0.69 - 0.31
        self.maxBrightness = 0.69 + 0.31
        self.minSaturation = -0.22 - 0.6
        self.maxSaturation = 0.22 + 0.6
        
#     read unprocessed parameters from json files
    def read_data_from_json(self):
        for sl in self.subject_list:
            with open("./data/pilot_study/json/" + sl + "_hue.json") as f:
                self.df_list_hue.append(json_normalize(json.load(f)))
            with open("./data/pilot_study/json/" + sl + "_nohue.json") as f:
                self.df_list_nohue.append(json_normalize(json.load(f)))

    
    # in order to complement "sliderHue" of '0'
    def calc_hsb_nohue(self):
        for i in range(len(self.df_list_nohue)):
            for j in range(self.order_num):
                red = int((255 * (self.df_list_nohue[i][str(j) + ".arousal"]  + 1)) / 2)
                green = 0
                blue = 255 - red
                # normalization
                red /= 255
                green /= 255
                blue /= 255
                # convert rgb to hue
                max_value = 0
                min_value = 0
                max_value = max(red, green, blue)
                min_value = min(red, green, blue)
                if max_value == min_value:
                    return
                elif min_value == blue:
                    self.df_list_nohue[i][str(j) + ".sliderHue"] = 60 * ((green - red) / (max_value - min_value)) + 60
                elif min_value == red:
                    self.df_list_nohue[i][str(j) + ".sliderHue"] = 60 * ((blue - green) / (max_value - min_value)) + 180
                elif min_value == green:
                    self.df_list_nohue[i][str(j) + ".sliderHue"] = 60 * ((red - blue) / (max_value - min_value)) + 300
#                 self.df_list_nohue[i][str(j) + ".sliderBrightness"] = ((self.df_list_nohue[i][str(j) + ".valence"] * 0.69 +
#                   self.df_list_nohue[i][str(j) + ".arousal"] * (-0.31) -
#                   self.minBrightness) / (self.maxBrightness - self.minBrightness)) * 100;
#                 self.df_list_nohue[i][str(j) + ".sliderSaturation"] = ((self.df_list_nohue[i][str(j) + ".valence"] * 0.22 +
#                   self.df_list_nohue[i][str(j) + ".arousal"] * 0.6 -
#                   self.minSaturation) /	(self.maxSaturation - self.minSaturation)) * 100;

    # concat all parameter for each characteristic of hue
    def concat_data_from_list(self):
        # hue
        self.overall_hue = pd.concat(self.df_list_hue, axis='rows')
        # nohue
        self.overall_nohue = pd.concat(self.df_list_nohue, axis='rows')
    
    def select_order(self):
        # make df_list whose shape have 17 types
        for i in range(self.order_num):
            self.df_each_order_hue.append(pd.DataFrame(index=self.parameter, columns=self.subject_list))
            self.df_each_order_nohue.append(pd.DataFrame(index=self.parameter, columns=self.subject_list))
            for j, param in enumerate(self.parameter):
                self.df_each_order_hue[i].iloc[j] = self.overall_hue[str(i) + "." + param].values
                self.df_each_order_nohue[i].iloc[j] = self.overall_nohue[str(i) + "." + param].values
        
    def select_parameter(self):
        self.df_each_parameter_hue = pd.DataFrame(index=[param for param in self.parameter], 
                                                  columns=["order_" + str(i) + "-" + str(j) for i in range(self.order_num) for j in range(len(self.subject_list))])
        self.df_each_parameter_nohue = pd.DataFrame(index=[param for param in self.parameter], 
                                                  columns=["order_" + str(i) + "-" + str(j) for i in range(self.order_num) for j in range(len(self.subject_list))])
        # make df_list whose shape have 17 types
        for param in self.parameter:
            for i in range(self.order_num):
                for j in range(len(self.subject_list)):
                    self.df_each_parameter_hue.loc[param, "order_" + str(i) + "-" + str(j)] = self.overall_hue[str(i) + "." + param].iloc[j]
                    self.df_each_parameter_nohue.loc[param, "order_" + str(i) + "-" + str(j)] = self.overall_nohue[str(i) + "." + param].iloc[j]
        
        
    def select_order_average(self):
        self.df_each_order_average_hue = pd.DataFrame(index=self.parameter, 
                                                      columns=["order_" + str(i) for i in range(self.order_num)])
        import statistics
        for param in self.parameter:
            for i in range(self.order_num):
                self.df_each_order_average_hue.loc[param, ["order_" + str(i)]] = "%.4f" % statistics.mean(self.df_each_order_hue[i].loc[param].values)
#         self.df_each_order_average_hue.to_csv("./df_each_order_average_hue.csv")
                
                
    def save_parameter_as_csv(self):
        # each parameter of subjects
        for i, sl in enumerate(self.subject_list):
            self.df_list_hue[i].to_csv("./data/pilot_study/csv/each_subject/" + sl + "_hue.csv")
            self.df_list_nohue[i].to_csv("./data/pilot_study/csv/each_subject/" + sl + "_nohue.csv")

        for i in range(self.order_num):
            self.df_each_order_hue[i].to_csv("./data/pilot_study/csv/each_order/" + "order_" + str(i) + "_hue.csv")
            self.df_each_order_nohue[i].to_csv("./data/pilot_study/csv/each_order/" + "order_" + str(i) + "_nohue.csv")
            
        # each parameter of order
        self.df_each_parameter_hue.to_csv("./data/pilot_study/csv/each_parameter/df_each_parameter_hue.csv")
        self.df_each_parameter_nohue.to_csv("./data/pilot_study/csv/each_parameter/df_each_parameter_nohue.csv")
        # overall parameter
        self.overall_nohue.to_csv("./data/pilot_study/csv/overall_nohue.csv")
        self.overall_hue.to_csv("./data/pilot_study/csv/overall_hue.csv")
        
    def save_order_as_xlsx(self):
        # save each parameter of subjects to one file (to some sheets)
        with pd.ExcelWriter("./data/pilot_study/csv/each_order/order_all_hue.xlsx") as w:
            for i, order in enumerate(self.df_each_order_hue):
                order.to_excel(w, sheet_name="order_" + str(i))
        with pd.ExcelWriter("./data/pilot_study/csv/each_order/order_all_nohue.xlsx") as w:
            for i, order in enumerate(self.df_each_order_nohue):
                order.to_excel(w, sheet_name="order_" + str(i))

In [3]:
if __name__ == '__main__':
    main = Main()
    main.read_data_from_json()
    main.calc_hsb_nohue()
    main.concat_data_from_list()
    main.select_order()
    main.select_parameter()
    main.select_order_average()
    main.save_parameter_as_csv()
#     main.save_order_as_xlsx()
